# Test for Each LOS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Reshape, Dropout, Attention, Concatenate, LayerNormalization, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score
from scipy.spatial.distance import cdist

# ---------------------- LOS-specific Config ----------------------
# LOS, a, window size, loss function
LOS_CONFIG = {
    "A": {"a": 0.3,  "window_size": 15, "loss": "mse"},
    "B": {"a": 0.5, "window_size": 15, "loss": "mse"},
    "C": {"a": 0.3,  "window_size": 15, "loss": "mae"},
    "D": {"a": 0.2, "window_size": 15, "loss": "mse"},
    "E": {"a": 0.6,  "window_size": 15, "loss": "mse"},
}

# ---------------------- Model Definition ----------------------

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        return true_positives / (possible_positives + K.epsilon())

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        return true_positives / (predicted_positives + K.epsilon())

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

def compute_residuals(y_true, y_pred, loss_type='rmse'):
    if loss_type == 'rmse':
        return np.sqrt(np.mean((y_true - y_pred) ** 2, axis=1))
    elif loss_type == 'mse':
        return np.mean((y_true - y_pred) ** 2, axis=1)
    elif loss_type == 'mae':
        return np.mean(np.abs(y_true - y_pred), axis=1)
    elif loss_type == 'huber':
        delta = 1.0
        diff = y_true - y_pred
        return np.mean(np.where(np.abs(diff) <= delta,
                                0.5 * diff ** 2,
                                delta * (np.abs(diff) - 0.5 * delta)), axis=1)
    elif loss_type == 'hausdorff':
        return np.array([
            hausdorff_distance(np.expand_dims(y_true[i], axis=0), np.expand_dims(y_pred[i], axis=0))
            for i in range(len(y_true))
        ])
    else:
        raise ValueError(f"Unknown loss type: {loss_type}")

def hausdorff_distance(set1, set2):
    dists = cdist(set1, set2, metric='euclidean')
    forward = np.max(np.min(dists, axis=1))
    backward = np.max(np.min(dists, axis=0))
    return max(forward, backward)

def build_prediction_model(input_shape, output_steps, n_features):
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Bidirectional(GRU(64, return_sequences=True))(x)

    attention = Attention()([x, x])  # Self-attention
    x = Concatenate()([x, attention])
    x = LayerNormalization()(x)
    x = Dropout(0.1)(x)

    x = GRU(64)(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(output_steps * n_features)(x)
    outputs = Reshape((output_steps, n_features))(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.00093286), loss='mse', metrics=['mae', f1])
    return model

def load_trained_model(input_shape, output_steps, n_features):
    model = build_prediction_model(input_shape, output_steps, n_features)
    model.load_weights("gru70_epoch_100.weights.h5")
    return model

# ---------------------- Helper Functions ----------------------

def calculate_entropy(data):
    value_counts = np.bincount(data)
    probabilities = value_counts / np.sum(value_counts)
    probabilities = probabilities[probabilities > 0]
    return -np.sum(probabilities * np.log2(probabilities))

def dynthreshold(vectime, vecvalue, window_size=15, a=0.2):
    """
    Entropy-based dynamic threshold.
    Parameters:
        vectime: not used in entropy calc but kept for interface completeness.
        vecvalue: series of values (will be rounded to int for entropy bins).
        window_size: rolling window for entropy calculation.
        a: scaling on std-dev to set threshold T1 = H_avg + a * sigma.
    """
    packets_int = np.round(vecvalue).astype(int)
    num_points = len(packets_int)
    entropy_values = np.array([
        calculate_entropy(packets_int[max(0, i - window_size):i + 1]) for i in range(num_points)
    ])
    H_avg = np.mean(entropy_values)
    sigma = np.std(entropy_values)
    T_1 = H_avg + a * sigma
    return entropy_values, T_1, H_avg

def test_single_sequence(model, input_sequence):
    input_seq = np.expand_dims(input_sequence, axis=0)
    prediction = model.predict(input_seq, verbose=0)[0]
    return prediction

def compute_ewma(series, alpha=0.2):
    ewma = np.zeros_like(series)
    ewma[0] = series[0]
    for t in range(1, len(series)):
        ewma[t] = alpha * series[t] + (1 - alpha) * ewma[t - 1]
    return ewma

# ---------------------- Main Processing ----------------------

if __name__ == "__main__":
    df = pd.read_csv('sequence_data_with_jerk_Attack_70.csv')

    look_back = 10
    look_forward = 5
    n_features = 3
    feature_names = ['Speed', 'Acceleration', 'Jerk']
    target_feature = 'Jerk'

    model = load_trained_model(
        input_shape=(look_back, n_features),
        output_steps=look_forward,
        n_features=n_features
    )

    sequences = df[['LOS', 'DepartureTime', 'Iteration']].drop_duplicates()
    metrics_summary = []

    for idx, row in sequences.iterrows():
        start_time = time.time()

        los = row['LOS']
        departure_time = row['DepartureTime']
        iteration = row['Iteration']
        print(los, departure_time, iteration)

        # Per-LOS configuration (fallback provided just in case)
        cfg = LOS_CONFIG.get(str(los), {"a": 0.2, "window_size": 15, "loss": "rmse"})
        a_val = cfg["a"]
        win_size = cfg["window_size"]
        loss_type = cfg["loss"]

        # Extract this sequence
        seq_df = df[(df['LOS'] == los) & (df['DepartureTime'] == departure_time) & (df['Iteration'] == iteration)]
        if len(seq_df) < (look_back + look_forward):
            continue

        time_sequence = seq_df['Time'].values
        full_features = seq_df[['Speed', 'Acceleration', 'Jerk']].values
        full_labels = seq_df['Label'].values.astype(int)

        all_predictions, all_actuals, all_times, all_labels = [], [], [], []

        for i in range(0, len(full_features) - look_back - look_forward + 1, look_forward):
            current_window = full_features[i:i+look_back]
            actual_continuation = full_features[i+look_back:i+look_back+look_forward]
            time_window = time_sequence[i+look_back:i+look_back+look_forward]
            label_window = full_labels[i+look_back:i+look_back+look_forward]
            prediction = test_single_sequence(model, current_window)
            all_predictions.append(prediction)
            all_actuals.append(actual_continuation)
            all_times.append(time_window)
            all_labels.append(label_window)

        if not all_predictions:
            continue

        all_predictions = np.concatenate(all_predictions)
        all_actuals = np.concatenate(all_actuals)
        all_times = np.concatenate(all_times)
        all_labels = np.concatenate(all_labels)

        predicted_sequence = np.vstack([full_features[:look_back], all_predictions])
        actual_sequence = np.vstack([full_features[:look_back], all_actuals])
        time_plot = np.concatenate([time_sequence[:look_back], all_times])
        test_labels = np.concatenate([full_labels[:look_back], all_labels])

        # ---- LOS-specific residuals & anomaly scoring ----
        feature_idx = feature_names.index(target_feature)

        # Use only the target feature for residuals; shape to (N,1) to match compute_residuals' axis=1 behavior
        y_true_seq = actual_sequence[:, feature_idx].reshape(-1, 1)
        y_pred_seq = predicted_sequence[:, feature_idx].reshape(-1, 1)

        residual_series = compute_residuals(y_true_seq, y_pred_seq, loss_type=loss_type)

        # Smooth (discard the initial look_back to align with predictions like before)
        smoothed_residuals = compute_ewma(residual_series[look_back:])

        # Dynamic threshold with LOS-specific 'a' and entropy window
        entropy_vals, T1, H_avg = dynthreshold(time_plot[look_back:], smoothed_residuals,
                                               window_size=win_size, a=a_val)
        anomalies = (entropy_vals > T1).astype(int)

        # Align labels
        y_true = test_labels[look_back:look_back+len(smoothed_residuals)]
        y_pred = anomalies

        # Metrics
        TP = np.sum((y_pred == 1) & (y_true == 1))
        FP = np.sum((y_pred == 1) & (y_true == 0))
        FN = np.sum((y_pred == 0) & (y_true == 1))
        TN = np.sum((y_pred == 0) & (y_true == 0))

        try:
            auc = roc_auc_score(y_true, entropy_vals)
        except ValueError:
            auc = np.nan

        f1_val = f1_score(y_true, y_pred, zero_division=0)
        prob_detection = (TP / (TP + FN)) * 100 if (TP + FN) > 0 else 0.0
        false_alarm_rate = (FP / (FP + TN)) * 100 if (FP + TN) > 0 else 0.0
        anomaly_probability = (np.sum(y_pred == 1) / len(y_pred)) if len(y_pred) > 0 else 0.0

        end_time = time.time()

        metrics_summary.append({
            'LOS': los,
            'DepartureTime': departure_time,
            'Iteration': iteration,
            'LossType': loss_type,
            'EntropyWin': win_size,
            'Alpha_a': a_val,
            'TP': TP,
            'FP': FP,
            'FN': FN,
            'TN': TN,
            'PD': prob_detection,
            'FAR': false_alarm_rate,
            'F1': f1_val,
            'AUC': auc,
            'Anomaly_Prob': anomaly_probability,
            'Time_sec': end_time - start_time
        })

        # ---- Progress summary (writes each pass so you can monitor long runs) ----
        if metrics_summary:
            metrics_df = pd.DataFrame(metrics_summary)
            numeric_cols = metrics_df.select_dtypes(include=np.number).columns

            print(f"\n=== Summary Metrics for Feature: {target_feature} ===")
            print(f"Window Configuration: look_back={look_back}, look_forward={look_forward}")
            print(f"Total sequences analyzed: {len(metrics_df)}\n")

            grouped = metrics_df.groupby('LOS')[numeric_cols]

            median_metrics = grouped.median().round(4)
            mean_metrics = grouped.mean().round(4)

            print("Median Metrics by LOS:")
            print(median_metrics)

            print("\nMean Metrics by LOS:")
            print(mean_metrics)

            # Save outputs (now include loss info in filename prefix)
            median_metrics.to_csv(f"median_metrics_{target_feature}.csv")
            mean_metrics.to_csv(f"mean_metrics_{target_feature}.csv")
            metrics_df.to_csv(f"metrics_summary_{target_feature}.csv", index=False)
        else:
            print(f"No metrics collected for feature: {target_feature}")


/share/home2/kamtams/jintel/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 36 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


A 895 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1